## Compiling Pre-Trained Models from keras
In this notebook, we illustrate how we can use `nnexpress` to compile pre-trained models from the keras framework. The notebook illustrates how to load a model, convert it to tflite, save the tflite file, and compile it for n2x sw and hw options. The code is tested on multiple models which can be chosen by uncommenting the appropriate line.

In [ ]:
#imports for tensorflow and keras models
import tensorflow as tf
from tensorflow.keras.applications import MobileNet, MobileNetV2, MobileNetV3Small, MobileNetV3Large
from tensorflow.keras.applications import ResNet50, ResNet50V2, ResNet101, ResNet152, ResNet152V2
#imports for nnexpress compiler and utilities
import nnexpress.compiler as n2xc
from nnexpress_utils import run_n2x, compare_n2x_sw_hw, estimate_fps

In [ ]:
model=MobileNet(input_shape=(224,224,3),weights='imagenet',include_top=False, pooling='avg')
# model=MobileNetV2(input_shape=(224,224,3),weights='imagenet',)
# model=MobileNetV3Small(input_shape=(224,224,3),weights='imagenet',minimalistic=True, 
#                        include_top=False,pooling='avg', include_preprocessing=False)
# model=MobileNetV3Large(input_shape=(224,224,3),weights='imagenet',minimalistic=True, 
#                        include_top=False,pooling='avg', include_preprocessing=False)
# model=ResNet50(input_shape=(224,224,3),weights='imagenet',)
# model=ResNet50V2(input_shape=(224,224,3),weights='imagenet',include_top=False,pooling='avg')
# model=ResNet101(input_shape=(224,224,3),weights='imagenet',)
# model=ResNet152(input_shape=(224,224,3),weights='imagenet',)
# model=ResNet152V2(input_shape=(224,224,3),weights='imagenet',include_top=False,pooling='avg')

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) 
tflite_model = converter.convert()
# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
tflite_path='model.tflite'
n2x_sw_path='model_sw.n2x'
n2x_hw_path='model_hw.n2x'

In [ ]:
sw_model=n2xc.Compiler(tflite_path, device='SW')
hw_model=n2xc.Compiler(tflite_path, device='HW')
sw_model.save(n2x_sw_path)
hw_model.save(n2x_hw_path)

**Note**: This below cells can run only on a device equipped with ORCA

In [ ]:
match=compare_n2x_sw_hw(n2x_sw_path,n2x_hw_path)
if match:
    print('N2X SW and HW match')
else:
    print('N2X SW and HW do not match')    

In [ ]:
fps=estimate_fps(n2x_hw_path)
print('FPS=',fps)